**Importing Libraries**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

#importing deep learning libraries
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

**Extracting Training data & Validation data**

In [2]:
datagen_train = ImageDataGenerator()

training_set = datagen_train.flow_from_directory(r"C:\Users\ritul\OneDrive\Documents\ML projects\Main Projects\Emotion Detector\data\train",
                                                target_size=(48,48),
                                                color_mode="grayscale",
                                                batch_size=128,
                                                class_mode="categorical",
                                                shuffle=True)



Found 28709 images belonging to 7 classes.


In [3]:
datagen_val = ImageDataGenerator()
test_set = datagen_train.flow_from_directory(r"C:\Users\ritul\OneDrive\Documents\ML projects\Main Projects\Emotion Detector\data\validation",
                                                target_size=(48,48),
                                                color_mode="grayscale",
                                                batch_size=128,
                                                class_mode="categorical",
                                                shuffle=True)

Found 7178 images belonging to 7 classes.


**Creating the Emotion Detector model**

In [4]:
no_of_classes = 7
model = Sequential()

# First CNN layer
model.add(Conv2D(filters=64, kernel_size=3, activation='relu',input_shape=(48,48,1),padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25)) #prevents overfitting

# Second CNN layer
model.add(Conv2D(filters=128, kernel_size=3, activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25)) 

# Third CNN layer
model.add(Conv2D(filters=512, kernel_size=3, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25)) 

# Fourth CNN layer
model.add(Conv2D(filters=512, kernel_size=3, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25)) 

# Flattening
model.add(Flatten())

# Fully connected first layer
model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

# Fully connected second layer
model.add(Dense(64,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))


model.add(Dense(no_of_classes, activation='softmax'))

In [5]:
#Compiling the CNN
opt = Adam(learning_rate= 0.0001) 
model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=['accuracy'])

In [6]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 64)        640       
                                                                 
 batch_normalization (BatchN  (None, 48, 48, 64)       256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 24, 24, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 128)       73856     
                                                                 
 batch_normalization_1 (Batc  (None, 24, 24, 128)      5

**Fitting the model with training & validation data**

In [7]:
checkpoint = ModelCheckpoint("./model.h5", monitor='val_acc', verbose=1, save_best_only=True, mode='max')

early_stopping = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
                          )

reduce_learningrate = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

callbacks_list = [early_stopping,checkpoint,reduce_learningrate]


In [16]:
batch_size = 32
history = model.fit_generator(generator=training_set, 
                                epochs=10, 
                                validation_data=test_set, 
                                callbacks=callbacks_list)


Epoch 1/10


C:\Users\ritul\AppData\Local\Temp\ipykernel_24396\2585796990.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(generator=training_set,


225/225 [==============================] - 514s 2s/step - loss: 2.0191 - accuracy: 0.3058 - val_loss: 1.6323 - val_accuracy: 0.3945 - lr: 1.0000e-04
Epoch 2/10
225/225 [==============================] - 515s 2s/step - loss: 1.9121 - accuracy: 0.3374 - val_loss: 1.5822 - val_accuracy: 0.4067 - lr: 1.0000e-04
Epoch 3/10
225/225 [==============================] - 517s 2s/step - loss: 1.8233 - accuracy: 0.3625 - val_loss: 1.5129 - val_accuracy: 0.4312 - lr: 1.0000e-04
Epoch 4/10
225/225 [==============================] - 519s 2s/step - loss: 1.7598 - accuracy: 0.3863 - val_loss: 1.4960 - val_accuracy: 0.4457 - lr: 1.0000e-04
Epoch 5/10
225/225 [==============================] - 578s 3s/step - loss: 1.6982 - accuracy: 0.4020 - val_loss: 1.4762 - val_accuracy: 0.4563 - lr: 1.0000e-04
Epoch 6/10
225/225 [==============================] - 585s 3s/step - loss: 1.6258 - accuracy: 0.4229 - val_loss: 1.4174 - val_accuracy: 0.4720 - lr: 1.0000e-04
Epoch 7/10
225/225 [==============================]